# Importing 

In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Loading the datasets

In [2]:
coding = pd.read_csv("mouse_coding_9_features.csv")
noncoding = pd.read_csv("mouse_noncoding_9_features.csv")

In [4]:
#replace ORF ratio
coding["ORF Ratio"] = coding["ORF Length"] / coding["Transcript Length"]

#add new feature
coding["CpG Islands per ORF Length"] = coding["CpG Islands"] / coding["ORF Length"]

#add new feature
coding["GC content per ORF Length"] = coding["GC content"] / coding["ORF Length"]

#adding labels
coding["coding/noncoding"] = 1

print(coding.shape)

(66799, 12)


In [22]:
#replace ORF ratio
noncoding["ORF Ratio"] = noncoding["ORF Length"] / noncoding["Transcript Length"]

#add new feature
noncoding["CpG Islands per ORF Length"] = noncoding["CpG Islands"] / noncoding["ORF Length"]

#add new feature
noncoding["GC content per ORF Length"] = noncoding["GC content"] / noncoding["ORF Length"]

#adding labels
noncoding["coding/noncoding"] = 0

print(noncoding.shape)

(26400, 12)


In [23]:
# taking 26400 sequences from both dataframes
noncoding = noncoding.head(26400)
coding = coding.head(26400)

# combining both dataframes
dataset = coding.merge(noncoding, how = "outer")

# shuffle the dataset
shuffled_dataset = dataset.sample(frac=1, random_state = 42)

In [24]:
print(shuffled_dataset.isna().sum())

Fickett Score                  0
GC content                     0
CpG Islands                    0
Transcript Length              0
ORF Length                     0
ORF Ratio                      0
Relative Codon Bias            0
Isoelectric Potential          0
Aromaticity                    0
coding/noncoding               0
CpG Islands per ORF Length    23
GC content per ORF Length      0
dtype: int64


In [25]:
import numpy as np
print(np.isinf(shuffled_dataset).sum())

Fickett Score                   0
GC content                      0
CpG Islands                     0
Transcript Length               0
ORF Length                      0
ORF Ratio                       0
Relative Codon Bias             0
Isoelectric Potential           0
Aromaticity                     0
coding/noncoding                0
CpG Islands per ORF Length    493
GC content per ORF Length     516
dtype: int64


In [26]:
shuffled_dataset.replace([np.inf, -np.inf], 0, inplace=True)

In [27]:
shuffled_dataset.fillna(0, inplace=True)

In [28]:
print(shuffled_dataset.isna().sum())

Fickett Score                 0
GC content                    0
CpG Islands                   0
Transcript Length             0
ORF Length                    0
ORF Ratio                     0
Relative Codon Bias           0
Isoelectric Potential         0
Aromaticity                   0
coding/noncoding              0
CpG Islands per ORF Length    0
GC content per ORF Length     0
dtype: int64


# Logistic Regression

## F5 - ORF len + FS + ORF Ratio + GCC + trans len + RCB + AR + IP + CpG/ORF +GC/ORF + CpG

In [37]:
X = shuffled_dataset.drop("coding/noncoding", axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# create the logistic regression classifier and fit it to the training data
clf = LogisticRegression(random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.8803977272727272


/home/user/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9228003384094755
Recall:  0.8293100171070139
F1 Score:  0.8735609170087096


## F4 - ORF len + FS + AR + IP + CpG/ORF + GC/ORF + CpG

In [32]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","ORF Ratio","GC content"], axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target


# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# create the logistic regression classifier and fit it to the training data
clf = LogisticRegression(random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Accuracy: 0.8806818181818182


/home/user/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9239245602881967
Recall:  0.8287397833111576
F1 Score:  0.87374749498998


## F3 - ORF len + FS + ORF Ratio + AR + IP + CpG/ORF + GC/ORF + CpG

In [40]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","GC content"], axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the logistic regression classifier and fit it to the training data
clf = LogisticRegression(random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Accuracy: 0.8794507575757575


/home/user/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9229953330504879
Recall:  0.8270290819235887
F1 Score:  0.8723809523809523


## F2 - ORF len + FS + ORF Ratio + CpG/ORF + GC/ORF + CpG

In [38]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","GC content","Aromaticity","Relative Codon Bias","Isoelectric Potential"], axis=1)  # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the logistic regression classifier and fit it to the training data
clf = LogisticRegression(random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



Accuracy: 0.8783143939393939


In [39]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9204737732656514
Recall:  0.8272191598555407
F1 Score:  0.8713584943437781


## F1 - ORF len + FS + ORF Ratio + CpG/ORF + GC/ORF

In [41]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","CpG Islands","Transcript Length","GC content","Aromaticity","Relative Codon Bias","Isoelectric Potential"], axis=1)   # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the logistic regression classifier and fit it to the training data
clf = LogisticRegression(random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


Accuracy: 0.8782196969696969


In [42]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9193922768516565
Recall:  0.8281695495153013
F1 Score:  0.8714000000000002


# Random Forest

## F1 - ORF len + FS + ORF Ratio + CpG/ORF + GC/ORF

In [45]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","CpG Islands","Transcript Length","GC content","Aromaticity","Relative Codon Bias","Isoelectric Potential"], axis=1)   # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Accuracy: 0.8929924242424242


In [46]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9115361625821877
Recall:  0.8696065386808591
F1 Score:  0.8900778210116732


## F2 - ORF len + FS + ORF Ratio + CpG/ORF + GC/ORF + CpG

In [47]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","GC content","Aromaticity","Relative Codon Bias","Isoelectric Potential"], axis=1)  # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



Accuracy: 0.89375


In [50]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)


Precision:  0.9172
Recall:  0.8716973959323323
F1 Score:  0.8938699931780529


## F3 - ORF len + FS + ORF Ratio + AR + IP + CpG/ORF + GC/ORF + CpG

In [49]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","GC content"], axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



Accuracy: 0.896875


In [51]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9172
Recall:  0.8716973959323323
F1 Score:  0.8938699931780529


## F4 - ORF len + FS + AR + IP + CpG/ORF + GC/ORF + CpG

In [52]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","ORF Ratio","GC content"], axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



Accuracy: 0.893560606060606


In [53]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9144460028050491
Recall:  0.8675156814293861
F1 Score:  0.8903628560280921


## F5 - ORF len + FS + ORF Ratio + GCC + trans len + RCB + AR + IP + CpG/ORF +GC/ORF + CpG

In [54]:
X = shuffled_dataset.drop("coding/noncoding", axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



Accuracy: 0.9025568181818182


In [55]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9243882872041717
Recall:  0.8760691883672306
F1 Score:  0.8995803649848736


# Naive Bayes 

## F5 - ORF len + FS + ORF Ratio + GCC + trans len + RCB + AR + IP + CpG/ORF +GC/ORF + CpG

In [56]:
X = shuffled_dataset.drop("coding/noncoding", axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = GaussianNB()
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Accuracy: 0.8262310606060606


In [57]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.92825
Recall:  0.7057593613381486
F1 Score:  0.8018572508368428


## F4 - ORF len + FS + AR + IP + CpG/ORF + GC/ORF + CpG

In [58]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","ORF Ratio","GC content"], axis=1) # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = GaussianNB()
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Accuracy: 0.8290719696969697


In [59]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9266666666666666
Recall:  0.7133624786162327
F1 Score:  0.80614327139942


## F3 - ORF len + FS + ORF Ratio + AR + IP + CpG/ORF + GC/ORF + CpG

In [60]:
X = shuffled_dataset.drop(["coding/noncoding","Relative Codon Bias","Transcript Length","GC content","Aromaticity","Relative Codon Bias","Isoelectric Potential"], axis=1)  # Features
Y = shuffled_dataset["coding/noncoding"] # Target

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# create the Random Forest classifier and fit it to the training data
clf = GaussianNB()
clf.fit(X_train, Y_train)

# predict the labels for the test set
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = accuracy_score(Y_test, Y_pred)

print('Accuracy:', accuracy)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Accuracy: 0.8357954545454546


In [61]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

f1 = f1_score(Y_test, Y_pred)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

Precision:  0.9319128092089151
Recall:  0.7232465310777418
F1 Score:  0.8144263698630136


## F2 - ORF len + FS + ORF Ratio + CpG/ORF + GC/ORF + CpG